In [208]:
import os
from os import path
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image, ImageDraw
from ply import read_ply
from tqdm.auto import tqdm, trange
import torch
import pandas as pd

In [209]:
from plotly import express as px
from plotly import io as pio

pio.templates.default = "plotly_dark"

In [238]:
CHECKPOINT_PATH = "yolov5/agro-hack-yolo/exp21/weights/best.pt"
DATA_PATH = "E:\\GENA_RT\\DATA\\agro_hack\\final_test"
CACHE_PATH = path.join(DATA_PATH, 'cache')
RESULTS_PATH = path.join(DATA_PATH, 'results')

chunks = ['20210901_152837_step_20',
          '20210929_092132_step_20',
          '20220719_185720_step_40',
          '20220720_101342_step_40',
          '20220724_143417_step_40'][1:]

In [211]:
def sort_by_suffix_number(arr):
    arr = [(int(e[e.index('_')+1:]), e) for e in arr]
    arr = sorted(arr)
    return [e[1] for e in arr]

In [212]:
def depth_map_from_ply(ply_path):
    points = read_ply(ply_path)['points'].to_numpy()

    cx = 240 #  160
    cy = len(points) // cx

    depth_map = np.zeros((cx, cy), dtype=float)
    for x in range(cx):
        for y in range(cy):
            depth_map[x][y] = np.linalg.norm(np.array(list(points[x * cy + y])))
            if depth_map[x][y] < 0.01:
                depth_map[x][y] = 15

    depth_map -= np.min(depth_map)
    depth_map = depth_map / np.max(depth_map)

    return depth_map

def swap_channel(rgb, depth, ch):
    rgb_np = np.array(rgb)
    depth_np = np.array(depth)
    rgb_np = np.transpose(rgb_np, (2, 0, 1))
    rgb_np[ch] = depth_np
    rgb_np = np.transpose(rgb_np, (1, 2, 0))
    return Image.fromarray(rgb_np)

def image_from_package(package_path):
    depth = depth_map_from_ply(path.join(package_path, 'point_cloud.ply'))
    depth = Image.fromarray(np.uint8(depth * 255) , 'L')
    depth = depth.resize((848, 480))

    rgb = Image.open(path.join(package_path, 'color_frame.png'))
    rgb = rgb.resize((848, 480))

    return swap_channel(rgb, depth, 2)

In [213]:
STEP = 1
REWRITE = True

for chunk_i, chunk in enumerate(chunks):
    cache_save_path = path.join(CACHE_PATH, chunk)
    if path.exists(cache_save_path):
        if not REWRITE:
            continue
        else:
            shutil.rmtree(cache_save_path)
    os.makedirs(cache_save_path)

    packages = os.listdir(path.join(DATA_PATH, chunk))
    packages = sort_by_suffix_number(packages)
    for package_i, package in enumerate(tqdm(packages[::STEP], desc=f'chunk {chunk_i+1}/{len(chunks)}')):
        image = image_from_package(path.join(DATA_PATH, chunk, package))
        image.save(path.join(cache_save_path, str(package_i).zfill(6) + '.jpg'))

chunk 1/1:   0%|          | 0/125 [00:00<?, ?it/s]

In [214]:
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model = torch.hub.load('ultralytics/yolov5', 'custom', path=CHECKPOINT_PATH)  # local model

# Image
im = 'test/images/20220724_171831-package_10000_png.png'

# Inference
results = model(im)

results.pandas().xyxy[0]

Using cache found in C:\Users\densh/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-18 Python-3.8.10 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


,xmin,ymin,xmax,ymax,confidence,class,name
0,0.0,116.283867,690.930542,427.349792,0.284762,0,object


In [244]:
BATCH_SIZE = 256

model.conf = 0.01
model.eval()
model.cuda()


info = []
df = []

for chunk_i, chunk in enumerate(chunks):
    chunk_results_path = path.join(RESULTS_PATH, chunk)
    if path.exists(chunk_results_path):
        shutil.rmtree(chunk_results_path)
    os.makedirs(chunk_results_path)

    images = os.listdir(path.join(CACHE_PATH, chunk))
    image_paths = [path.join(CACHE_PATH, chunk, image) for image in images]

    chunk_info = []

    for idx in trange((len(image_paths) - 1) // BATCH_SIZE + 1, desc=f'chunk {chunk_i+1}/{len(chunks)}'):
        batch = image_paths[idx * BATCH_SIZE: (idx + 1) * BATCH_SIZE]
        results = model(batch)

        chunk_info.extend(results.pandas().xywh)

        #results.render()
        for img, image_path, img_idx in zip(results.ims, batch, range(len(batch))):
            step = 20 if chunk.endswith('20') else 40
            package = (idx * BATCH_SIZE + img_idx) * step
            package_id = f'/{chunk}/package_{package}'

            objects = results.pandas().xywhn[img_idx]
            best = []
            best_value = 0
            for obj in objects.values:
                xc, yc, w, h, conf, _, _ = obj
                if w * h * conf > best_value:
                    best = [xc, yc, w, h]
                    best_value = w * h * conf
            best = [package_id] + best
            df.append(best)

            image_path = image_path.replace(CACHE_PATH, RESULTS_PATH)
            img = Image.fromarray(img)


    info.append(chunk_info)

chunk 1/4:   0%|          | 0/1 [00:00<?, ?it/s]

chunk 2/4:   0%|          | 0/1 [00:00<?, ?it/s]

chunk 3/4:   0%|          | 0/1 [00:00<?, ?it/s]

chunk 4/4:   0%|          | 0/1 [00:00<?, ?it/s]

In [246]:
pd_df = pd.DataFrame(df, columns=['package_id','board_point_x','board_point_y','width','height'],)
pd_df.head()

,package_id,board_point_x,board_point_y,width,height
0,/20210929_092132_step_20/package_0,0.302451,0.241510,0.604901,0.415279
1,/20210929_092132_step_20/package_20,0.277220,0.248038,0.554440,0.402017
2,/20210929_092132_step_20/package_40,0.265471,0.245764,0.530943,0.391328
3,/20210929_092132_step_20/package_60,0.272874,0.245061,0.545748,0.399801
4,/20210929_092132_step_20/package_80,0.291081,0.248336,0.577965,0.423472


In [247]:
pd_df.to_csv('second.csv', index=False)

In [216]:
info[0][0]

,xcenter,ycenter,width,height,confidence,class,name
0,561.734558,72.432213,572.530884,115.219460,0.021607,0,object
1,281.766937,61.888332,533.051208,92.933022,0.019170,0,object
2,316.621155,426.824188,632.618652,106.351624,0.016626,0,object
3,115.270905,276.007080,205.350159,41.116043,0.012142,0,object


In [217]:
TRAIN_IMAGES_PATH = 'train/images'
train_set = [[] for _ in chunks]

for image in os.listdir(TRAIN_IMAGES_PATH):
    chunk = image[:image.index('-')]
    package = image[image.index('-')+1: image.index('_png')]
    train_set[chunks.index(chunk)].append(int(package[package.index('_') + 1:]))


ValueError: '20220719_183951' is not in list

In [ ]:
train_set = list(map(lambda l: list(sorted(l)), train_set))

def find_distance(chunk_i, package):
    dist = 1e9
    for t_package in train_set[chunk_i]:
        dist = min(dist, abs(t_package - package))
    return dist

In [ ]:
PREDICTION_THRESHOLD = 0.1
IN_TRAIN_THRESHOLD = 10

for chunk_i, _ in enumerate(chunks):
    df = []
    for i, res in enumerate(info[chunk_i]):
        package = i * 10
        in_train = find_distance(chunk_i, package) < IN_TRAIN_THRESHOLD

        for j in range(len(res)):
            if res.confidence[j] > PREDICTION_THRESHOLD:
                df.append([res.confidence[j], len(res), package, in_train])
        if len(res) == 0:
            df.append([0, 0, package, in_train])

    df = pd.DataFrame(df, columns=['confidence', 'count', 'package', 'in_train'])

    #df.head()

    px.scatter(df, x='package', y='confidence', color='in_train', title=f'Chunk {chunk_i+1}').show()

In [ ]:
df

In [ ]:
package_number

In [ ]:
model.eval()
model.cpu()

In [ ]:
im = 'https://ultralytics.com/images/zidane.jpg'
model(torch.zeros(16,3,320,640))

In [248]:
df = pd.read_csv('DPMTeam.csv')
df.head()

,package_id,board_point_x,board_point_y,width,height
0,/20210901_152837_step_20/package_0,0.662423,0.150900,0.675154,0.240041
1,/20210901_152837_step_20/package_20,0.470290,0.426742,0.455736,0.221560
2,/20210901_152837_step_20/package_40,0.476865,0.463949,0.501136,0.248610
3,/20210901_152837_step_20/package_60,0.419723,0.510827,0.601290,0.326172
4,/20210901_152837_step_20/package_80,0.367799,0.490305,0.715931,0.283628


In [274]:
for row in tqdm(df.values):

    package, xc, yc, w, h = row
    img_path = path.join(DATA_PATH, package[1:], 'color_frame.png')
    img = Image.open(img_path)
    img = img.resize((848, 480))

    draw = ImageDraw.Draw(img)
    x1, y1, x2, y2 = xc - w/2, yc - h/2, xc + w/2, yc + h/2
    x1, y1, x2, y2 = x1 * img.width, y1 * img.height, x2 * img.width, y2 * img.height

    draw.rectangle((x1, y1, x2, y2), outline='red', width=4)


    chunk = package[1: package.index('/p')]
    chunk_results_path = path.join(DATA_PATH, 'render', chunk)
    #if path.exists(chunk_results_path):
        #shutil.rmtree(chunk_results_path)
    #os.makedirs(chunk_results_path)

    step = 20 if chunk.endswith('20') else 40
    package_num = int(package[package.rindex('_')+1:])
    img.save(path.join(chunk_results_path, str(package_num // step).zfill(6) + '.jpg'))


